In [1]:
import json
from scipy.interpolate import interp1d
from scipy.spatial.transform import Rotation, Slerp
import numpy as np
"""
[
    [
     0.   ts,
     1.   {"a":{
              "x",
              "y",
              "z"},
          "g":{
              "x",
              "y",
              "z"}
        },
      2.  {"x", "y", "z"},
      3.  {"x", "y", "z", "w"}   
    ],
    [...],
...]
"""

'\n[\n    [\n     0.   ts,\n     1.   {"a":{\n              "x",\n              "y",\n              "z"},\n          "g":{\n              "x",\n              "y",\n              "z"}\n        },\n      2.  {"x", "y", "z"},\n      3.  {"x", "y", "z", "w"}   \n    ],\n    [...],\n...]\n'

In [2]:
# coeffs for rotor speeds to thrust conversion
SCALE_THRUST_COEFF = -1.0
MASS = 0.915
C_T = 2.03e-8

# the provided ground truth is the drone body in the NED vicon frame
# rotate to have z upwards
R_w_ned = np.array([
    [1., 0., 0.],
    [0., -1., 0.],
    [0., 0., -1.]])
t_w_ned = np.array([0., 0., 0.])

# rotate from body to imu frame
R_b_i = np.array([
    [0., -1., 0.],
    [1., 0., 0.],
    [0., 0., 1.]])
t_b_i = np.array([0., 0., 0.])

jsobj = []
with open("19.02.2024.17_10_34.json") as fp:
    jsobj = json.load(fp)

# Read data
raw_imu = []  # [ts wx wy wz ax ay az]
thrusts = []  # [ts 0. 0. mass_normalized_thrust]

data = [] #gt

for dati in jsobj:
    raw_imu.append(np.array([dati[0],
                             dati[1]['g']['x'],
                             dati[1]['g']['y'],
                             dati[1]['g']['z'],
                             dati[1]['a']['x'],
                             dati[1]['a']['y'],
                             dati[1]['a']['z']]))

    omega = (15000 ** 2)*4
    thr = omega * (C_T / MASS)
    thr = SCALE_THRUST_COEFF * thr
    thrusts.append(np.array([dati[0], 0., 0., thr]))

    t_i = [dati[2]["x"], dati[2]["y"], dati[2]["z"]]
    R_i = Rotation.from_quat(
        np.array([dati[3]["x"], dati[3]["y"], dati[3]["z"], dati[3]["w"]])).as_matrix()

    # transform to world frame
    R_it = R_w_ned @ R_i
    t_it = t_w_ned + R_w_ned @ t_i

    # transform to imu frame
    t_it = t_it + R_it @ t_b_i
    R_it = R_it @ R_b_i

    q_it = Rotation.from_matrix(R_it).as_quat()
    d = np.array([
        dati[0],
        t_it[0], t_it[1], t_it[2],
        q_it[0], q_it[1], q_it[2], q_it[3]
    ])
    data.append(d)

data = np.asarray(data)
raw_imu = np.asarray(raw_imu)
thrusts = np.asarray(thrusts)

In [3]:
# include velocities
gt_times = data[:, 0]
gt_pos = data[:, 1:4]

# compute velocity
v_start = ((gt_pos[1] - gt_pos[0]) / (gt_times[1] - gt_times[0])).reshape((1, 3))
gt_vel_raw = (gt_pos[1:] - gt_pos[:-1]) / (gt_times[1:] - gt_times[:-1])[:, None]
gt_vel_raw = np.concatenate((v_start, gt_vel_raw), axis=0)
# filter
gt_vel_x = np.convolve(gt_vel_raw[:, 0], np.ones(5) / 5, mode='same')
gt_vel_x = gt_vel_x.reshape((-1, 1))
gt_vel_y = np.convolve(gt_vel_raw[:, 1], np.ones(5) / 5, mode='same')
gt_vel_y = gt_vel_y.reshape((-1, 1))
gt_vel_z = np.convolve(gt_vel_raw[:, 2], np.ones(5) / 5, mode='same')
gt_vel_z = gt_vel_z.reshape((-1, 1))
gt_vel = np.concatenate((gt_vel_x, gt_vel_y, gt_vel_z), axis=1)

gt_traj_tmp = np.concatenate((data, gt_vel), axis=1)  # [ts x y z qx qy qz qw vx vy vz]

# In Blackbird dataset, the sensors measurements are at:
# 100 Hz IMU meas.
# 180 Hz RPM meas.
# 360 Hz Vicon meas.
# resample imu at exactly 100 Hz
t_curr = raw_imu[0, 0]
dt = 0.01
new_times_imu = [t_curr]
while t_curr < raw_imu[-1, 0] - dt - 0.001:
    t_curr = t_curr + dt
    new_times_imu.append(t_curr)
new_times_imu = np.asarray(new_times_imu)
gyro_tmp = interp1d(raw_imu[:, 0], raw_imu[:, 1:4], axis=0)(new_times_imu)
accel_tmp = interp1d(raw_imu[:, 0], raw_imu[:, 4:7], axis=0)(new_times_imu)
raw_imu = np.concatenate((new_times_imu.reshape((-1, 1)), gyro_tmp, accel_tmp), axis=1)

# We down sample to IMU rate
times_imu = raw_imu[:, 0]
# get initial and final times for interpolations
idx_s = 0
for ts in times_imu:
    if ts > gt_traj_tmp[0, 0] and ts > thrusts[0, 0]:
        break
    else:
        idx_s = idx_s + 1
assert idx_s < len(times_imu)

idx_e = len(times_imu) - 1
for ts in reversed(times_imu):
    if ts < gt_traj_tmp[-1, 0] and ts < thrusts[-1, 0]:
        break
    else:
        idx_e = idx_e - 1
assert idx_e > 0

times_imu = times_imu[idx_s:idx_e + 1]
raw_imu = raw_imu[idx_s:idx_e + 1]

# interpolate ground-truth samples at thrust times
groundtruth_pos_data = interp1d(gt_traj_tmp[:, 0], gt_traj_tmp[:, 1:4], axis=0)(times_imu)
groundtruth_rot_data = Slerp(gt_traj_tmp[:, 0], Rotation.from_quat(gt_traj_tmp[:, 4:8]))(times_imu)
groundtruth_vel_data = interp1d(gt_traj_tmp[:, 0], gt_traj_tmp[:, 8:11], axis=0)(times_imu)

gt_traj = np.concatenate((times_imu.reshape((-1, 1)),
                          groundtruth_pos_data,
                          groundtruth_rot_data.as_quat(),
                          groundtruth_vel_data), axis=1)

# interpolate thrusts samples at imu times
thrusts_tmp = interp1d(thrusts[:, 0], thrusts[:, 1:4], axis=0)(times_imu)
thrusts = thrusts_tmp

ts = raw_imu[:, 0]


In [4]:
ts_test = ts
raw_imu_test = raw_imu
calib_imu_test = []
gt_traj_test = gt_traj
thrusts_test = thrusts
i_thrusts_test = thrusts_test

In [5]:
traj_target_oris_from_imu_list = []
traj_target_oris_from_imu_list.append(gt_traj[0])
traj_target_oris_from_imu = np.asarray(traj_target_oris_from_imu_list)

In [6]:
import h5py
def dict2arr(el):
    return np.array([el["x"], el["y"], el["z"]], dtype=np.float64)


ll = len(jsobj)
with h5py.File("mytestfile.hdf5", "w") as h5f:
    accel_bias = h5f.create_dataset("accel_bias", shape=(3,), dtype=np.float64)
    accel_bias[...] = np.zeros((3,))

    accel_calib = h5f.create_dataset("accel_calib", shape=(ll,3), dtype=np.float64)
    accel_calib[...] = np.array([dict2arr(jsobj[i][1]["a"]) for i in list(range(ll))])

    accel_raw = h5f.create_dataset("accel_raw", shape=(ll,3), dtype=np.float64)
    accel_raw[...] = np.array([dict2arr(jsobj[i][1]["a"]) for i in list(range(ll))])

    gyro_bias = h5f.create_dataset("gyro_bias", shape=(3,), dtype=np.float64)
    gyro_bias[...] = np.zeros((3,))

    gyro_calib = h5f.create_dataset("gyro_calib", shape=(ll,3), dtype=np.float64)
    gyro_calib[...] = np.array([dict2arr(jsobj[i][1]["g"]) for i in list(range(ll))])

    gyro_raw = h5f.create_dataset("gyro_raw", shape=(ll,3), dtype=np.float64)
    gyro_raw[...] = np.array([dict2arr(jsobj[i][1]["g"]) for i in list(range(ll))])

with h5py.File("my2.hdf5", "w") as f:
    ts = f.create_dataset("ts", data=ts_test)
    gyro_raw = f.create_dataset("gyro_raw", data=raw_imu_test[:, 1:4])
    accel_raw = f.create_dataset("accel_raw", data=raw_imu_test[:, 4:])
    gyro_calib = f.create_dataset("gyro_calib", data=raw_imu_test[:, 1:4])
    accel_calib = f.create_dataset("accel_calib", data=raw_imu_test[:, 4:])
    traj_target = f.create_dataset("traj_target", data=gt_traj_test[:, 1:8])
    traj_target_oris_from_imu_target = \
        f.create_dataset("traj_target_oris_from_imu", data=traj_target_oris_from_imu[:, 1:])
    thru = f.create_dataset("thrust", data=thrusts_test)
    i_thru = f.create_dataset("i_thrust", data=i_thrusts_test)
    gyro_bias = f.create_dataset("gyro_bias", data=np.zeros((3,)))
    accel_bias = f.create_dataset("accel_bias", data=np.zeros((3,)))
